In [1]:
import pandas as pd

In [2]:
df_all_features = pd.read_csv("apologyFullAfterScale.csv")

In [3]:
df_label = pd.read_csv("existence_label.csv")

In [4]:
df_dv = pd.DataFrame(df_label, columns = ["Label", "rating"])

In [5]:
df_merge = pd.concat([df_dv, df_all_features], axis = 1)

In [6]:
df_merge

,Label,rating,WC,Analytic,Clout,Authentic,Tone,WPS,Sixltr,Dic,...,Acknowledgement,Subjectivity,Bare.Command,YesNo.Questions,Gratitude,Apology,Truth.Intensifier,Affirmation,Adverb.Limiter,Conjunction.Start
0,1,30,-0.118588,1.146001,0.248374,0.025377,0.343271,-0.078194,-0.338293,0.354235,...,-0.266941,-0.119524,-0.199872,-0.07175,0.750244,0.313581,-0.208046,-0.157909,-0.284724,-0.390590
1,0,10,1.955549,0.315547,0.248374,-0.042275,-0.055201,0.463571,0.087842,-0.229171,...,3.091350,-0.119524,-0.199872,-0.07175,0.750244,-1.022993,-0.208046,-0.157909,-0.284724,-0.390590
2,1,30,-0.437686,0.505698,0.248374,-0.796046,0.343271,-0.441795,1.044277,-1.310556,...,-0.266941,-0.119524,-0.199872,-0.07175,-0.434352,0.313581,-0.208046,-0.157909,-0.284724,-0.390590
3,1,20,-0.158475,0.631818,0.248374,-0.869183,0.343271,-0.123644,-0.253066,0.566761,...,-0.266941,-0.119524,-0.199872,-0.07175,-0.434352,1.650154,-0.208046,-0.157909,-0.284724,-0.390590
4,1,30,0.280285,-0.141396,-0.522694,-0.449101,-4.661890,0.376307,-0.082612,-0.316682,...,-0.266941,-0.119524,-0.199872,-0.07175,0.750244,1.650154,-0.208046,-0.157909,-0.284724,-0.390590
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
775,1,10,0.260341,0.348533,-0.985844,1.655422,-0.238375,-0.110009,-0.789048,-1.158454,...,-0.266941,-0.119524,4.996794,-0.07175,-1.618947,-1.022993,-0.208046,-0.157909,-0.284724,-0.390590
776,1,30,3.969855,-0.421771,-1.342113,0.928621,-0.536375,0.217232,-0.603443,0.591765,...,-0.266941,8.355804,-0.199872,-0.07175,-1.618947,0.313581,-0.208046,-0.157909,5.799783,4.603844
777,1,20,-0.298080,0.155957,0.248374,-0.610917,-1.238997,1.398934,-0.432989,-0.985516,...,-0.266941,-0.119524,-0.199872,-0.07175,-1.618947,0.313581,-0.208046,-0.157909,-0.284724,-0.390590
778,1,10,-0.198362,-0.565354,0.248374,-0.935007,-1.427639,0.429029,0.553749,-0.789658,...,-0.266941,-0.119524,-0.199872,-0.07175,-0.434352,1.650154,-0.208046,-0.157909,-0.284724,-0.390590


### over sampling the no-apology case to create balance dataset (50% no-apology, 50% apology)

In [7]:
df_top_pos = df_merge[df_merge["Label"] == 1]
df_top_neg = df_merge[df_merge["Label"] == 0]

In [8]:
from sklearn.utils import resample
neg_upsample = resample(df_top_neg,
             replace=True,
             n_samples=len(df_top_pos),
             random_state=42)

print(neg_upsample.shape)

(519, 156)


In [9]:
data_upsampled = pd.concat([df_top_pos, neg_upsample])

In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC
from sklearn.metrics import plot_roc_curve
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt

---------

### Model uses word count as single variable 

In [11]:
X_data = data_upsampled[["WC"]]
y_data = data_upsampled['Label']
X_train, X_test, y_train, y_test = train_test_split(X_data,
                            y_data, stratify=y_data, #keep the same ratio
                            test_size=0.1,
                            random_state=42)

In [12]:
lgclf = LogisticRegression(random_state=42, fit_intercept = True, solver = "liblinear", penalty ="l1").fit(X_train, y_train) 

In [13]:
print(cross_val_score(lgclf, X_train, y_train, cv=10))

[0.65957447 0.55319149 0.62765957 0.63829787 0.65591398 0.61290323
 0.58064516 0.65591398 0.59139785 0.56989247]


In [14]:
print("Accuracy Score:",lgclf.score(X_test, y_test))

Accuracy Score: 0.6634615384615384


In [15]:
lgclf.predict(X_test)

array([1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1,
       1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0,
       0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1,
       0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1,
       0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0])

In [16]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, lgclf.predict(X_test))

array([[35, 17],
       [18, 34]])

In [17]:
#coef of wourd count
lgclf.coef_

array([[0.53846385]])

-----------

### Model using all features from text + politeness + text analyzer

In [18]:
features = data_upsampled.columns.to_list()[2:]

In [19]:
X_data = data_upsampled[features]
y_data = data_upsampled['Label']
X_train, X_test, y_train, y_test = train_test_split(X_data,
                            y_data, stratify=y_data, #keep the same ratio
                            test_size=0.1,
                            random_state=42)

In [24]:
lgclf = LogisticRegression(random_state=42, fit_intercept = True, solver = "liblinear", penalty ="l1").fit(X_train, y_train) 

#### what is the cross validation score?

In [46]:
print(cross_val_score(lgclf, X_train, y_train, cv=10).mean())

0.9185998627316403


#### what is the prediction accuracy of this model in testing dataset?

In [27]:
print("Accuracy Score:",lgclf.score(X_test, y_test))

Accuracy Score: 0.8942307692307693


#### what is the confusion matrix (true negative, false negative, false positive and true positive)

In [28]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, lgclf.predict(X_test))

array([[49,  3],
       [ 8, 44]])

In [29]:
lgclf.predict(X_test)

array([1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0,
       0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0,
       1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0])

In [30]:
coeff_df = pd.DataFrame(X_data.columns)
coeff_df.columns = ['Features']
coeff_df["Coefficient"] = pd.Series(lgclf.coef_[0])

# preview
coeff_df.sort_values(by='Coefficient', inplace=True)
df_f0 = coeff_df[coeff_df["Coefficient"] == 0]
df_feature = coeff_df[coeff_df["Coefficient"] != 0]

In [31]:
df_feature #features with prediction power

,Features,Coefficient
107,neg_sent_TA,-2.181692
122,Negation,-0.839138
40,cogproc,-0.832759
109,sadness_TA,-0.828771
86,Exclam,-0.791447
...,...,...
100,valence..EL._TA,0.914539
43,discrep,0.934765
22,negate,1.129916
34,sad,1.244119


In [32]:
df_f0 ## features with no prediction power

,Features,Coefficient
84,SemiC,0.0
80,AllPunc,0.0
148,Gratitude,0.0
79,filler,0.0
77,assent,0.0
128,Let.Me.Know,0.0
146,Bare.Command,0.0
74,death,0.0
117,they_TA,0.0
118,we_TA,0.0


In [62]:
df_topic = pd.read_csv("topic_model_ten_topics.csv")
df_topic = pd.DataFrame(df_topic, columns = ['t1', 't2', "t3", "t4","t5",'t6', 't7',"t8","t9","t10", 't11', 't12', "t13","t14","t15", 't16', 't17', "t18", "t19","t20"])

In [63]:
df_merge2 = pd.concat([df_merge, df_topic], axis = 1)

In [64]:
df_merge2 

,Label,rating,WC,Analytic,Clout,Authentic,Tone,WPS,Sixltr,Dic,...,t11,t12,t13,t14,t15,t16,t17,t18,t19,t20
0,1,30,-0.118588,1.146001,0.248374,0.025377,0.343271,-0.078194,-0.338293,0.354235,...,0.000027,0.000027,0.000027,0.000027,0.000027,0.000027,0.000027,0.999487,0.000027,0.000027
1,0,10,1.955549,0.315547,0.248374,-0.042275,-0.055201,0.463571,0.087842,-0.229171,...,0.000013,0.000013,0.000013,0.000013,0.444475,0.000013,0.000013,0.000013,0.000013,0.061649
2,1,30,-0.437686,0.505698,0.248374,-0.796046,0.343271,-0.441795,1.044277,-1.310556,...,0.000032,0.000032,0.999387,0.000032,0.000032,0.000032,0.000032,0.000032,0.000032,0.000032
3,1,20,-0.158475,0.631818,0.248374,-0.869183,0.343271,-0.123644,-0.253066,0.566761,...,0.000028,0.000028,0.000028,0.000028,0.000028,0.999472,0.000028,0.000028,0.000028,0.000028
4,1,30,0.280285,-0.141396,-0.522694,-0.449101,-4.661890,0.376307,-0.082612,-0.316682,...,0.000027,0.000027,0.000027,0.000027,0.000027,0.000027,0.000027,0.999487,0.000027,0.000027
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
775,1,10,0.260341,0.348533,-0.985844,1.655422,-0.238375,-0.110009,-0.789048,-1.158454,...,0.000021,0.000021,0.000021,0.000021,0.999604,0.000021,0.000021,0.000021,0.000021,0.000021
776,1,30,3.969855,-0.421771,-1.342113,0.928621,-0.536375,0.217232,-0.603443,0.591765,...,0.000009,0.000009,0.000009,0.000009,0.000009,0.000009,0.000009,0.000009,0.000009,0.000009
777,1,20,-0.298080,0.155957,0.248374,-0.610917,-1.238997,1.398934,-0.432989,-0.985516,...,0.000032,0.000032,0.000032,0.000032,0.000032,0.000032,0.000032,0.000032,0.000032,0.000032
778,1,10,-0.198362,-0.565354,0.248374,-0.935007,-1.427639,0.429029,0.553749,-0.789658,...,0.000029,0.000029,0.000029,0.000029,0.000029,0.000029,0.000029,0.000029,0.000029,0.000029


In [65]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler() 
topic_feature = df_merge2[['t1', 't2', "t3", "t4","t5",'t6', 't7',"t8","t9","t10", 't11', 't12', "t13","t14","t15", 't16', 't17', "t18", "t19","t20"]]
scale_feature = scaler.fit_transform(topic_feature)

In [66]:
df_scale_feature = pd.DataFrame(scale_feature, columns = ['t1', 't2', "t3", "t4","t5",'t6', 't7',"t8","t9","t10", 't11', 't12', "t13","t14","t15", 't16', 't17', "t18", "t19","t20"])

In [67]:
df_scale = pd.concat([df_merge2.iloc[:, :-20], df_scale_feature], axis = 1)

In [68]:
df_scale

,Label,rating,WC,Analytic,Clout,Authentic,Tone,WPS,Sixltr,Dic,...,t11,t12,t13,t14,t15,t16,t17,t18,t19,t20
0,1,30,-0.118588,1.146001,0.248374,0.025377,0.343271,-0.078194,-0.338293,0.354235,...,-0.204105,-0.235082,-0.269715,-0.134629,-0.163342,-0.508718,-0.135585,2.772350,-0.188538,-0.219639
1,0,10,1.955549,0.315547,0.248374,-0.042275,-0.055201,0.463571,0.087842,-0.229171,...,-0.204186,-0.235157,-0.269779,-0.134744,3.077054,-0.508756,-0.135695,-0.397852,-0.188622,0.101014
2,1,30,-0.437686,0.505698,0.248374,-0.796046,0.343271,-0.441795,1.044277,-1.310556,...,-0.204074,-0.235054,4.421057,-0.134585,-0.163304,-0.508704,-0.135543,-0.397792,-0.188506,-0.219612
3,1,20,-0.158475,0.631818,0.248374,-0.869183,0.343271,-0.123644,-0.253066,0.566761,...,-0.204101,-0.235078,-0.269712,-0.134623,-0.163336,2.232880,-0.135579,-0.397806,-0.188534,-0.219635
4,1,30,0.280285,-0.141396,-0.522694,-0.449101,-4.661890,0.376307,-0.082612,-0.316682,...,-0.204105,-0.235082,-0.269715,-0.134629,-0.163342,-0.508718,-0.135585,2.772349,-0.188538,-0.219639
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
775,1,10,0.260341,0.348533,-0.985844,1.655422,-0.238375,-0.110009,-0.789048,-1.158454,...,-0.204142,-0.235116,-0.269744,-0.134681,7.124413,-0.508735,-0.135635,-0.397828,-0.188576,-0.219671
776,1,30,3.969855,-0.421771,-1.342113,0.928621,-0.536375,0.217232,-0.603443,0.591765,...,-0.204214,-0.235183,-0.269802,-0.134785,-0.163477,-0.508769,-0.135734,-0.397867,-0.188652,-0.219735
777,1,20,-0.298080,0.155957,0.248374,-0.610917,-1.238997,1.398934,-0.432989,-0.985516,...,-0.204074,-0.235054,-0.269691,-0.134585,-0.163304,-0.508704,-0.135543,-0.397792,-0.188506,-0.219612
778,1,10,-0.198362,-0.565354,0.248374,-0.935007,-1.427639,0.429029,0.553749,-0.789658,...,-0.204091,-0.235069,-0.269704,-0.134609,-0.163325,-0.508712,-0.135565,-0.397801,-0.188524,-0.219627


In [72]:
df_top_pos = df_scale[df_scale["Label"] == 1]
df_top_neg = df_scale[df_scale["Label"] == 0]
neg_upsample = resample(df_top_neg,
             replace=True,
             n_samples=len(df_top_pos),
             random_state=42)
data_upsampled = pd.concat([df_top_pos, neg_upsample])

In [73]:
X_data = data_upsampled[features + ['t1', 't2', "t3", "t4","t5",'t6', 't7',"t8","t9","t10", 't11', 't12', "t13","t14","t15", 't16', 't17', "t18", "t19","t20"]]
y_data = data_upsampled['Label']
X_train, X_test, y_train, y_test = train_test_split(X_data,
                            y_data, stratify=y_data, #keep the same ratio
                            test_size=0.1,
                            random_state=42)

In [74]:
X_data 

,WC,Analytic,Clout,Authentic,Tone,WPS,Sixltr,Dic,function.,pronoun,...,t11,t12,t13,t14,t15,t16,t17,t18,t19,t20
0,-0.118588,1.146001,0.248374,0.025377,0.343271,-0.078194,-0.338293,0.354235,-0.295524,0.009033,...,-0.204105,-0.235082,-0.269715,-0.134629,-0.163342,-0.508718,-0.135585,2.772350,-0.188538,-0.219639
2,-0.437686,0.505698,0.248374,-0.796046,0.343271,-0.441795,1.044277,-1.310556,-0.485396,0.754838,...,-0.204074,-0.235054,4.421057,-0.134585,-0.163304,-0.508704,-0.135543,-0.397792,-0.188506,-0.219612
3,-0.158475,0.631818,0.248374,-0.869183,0.343271,-0.123644,-0.253066,0.566761,0.154079,0.390709,...,-0.204101,-0.235078,-0.269712,-0.134623,-0.163336,2.232880,-0.135579,-0.397806,-0.188534,-0.219635
4,0.280285,-0.141396,-0.522694,-0.449101,-4.661890,0.376307,-0.082612,-0.316682,1.031791,-0.010709,...,-0.204105,-0.235082,-0.269715,-0.134629,-0.163342,-0.508718,-0.135585,2.772349,-0.188538,-0.219639
6,-0.118588,0.527041,0.248374,-0.466928,0.343271,1.807985,1.260185,-0.650057,-0.727215,-0.254192,...,-0.204117,-0.235093,-0.269725,-0.134646,-0.163357,-0.508724,-0.135601,-0.397815,-0.188551,-0.219650
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
654,-0.078701,-0.193299,-1.507524,-0.521781,-0.493998,-0.032744,0.695794,-2.791991,-1.357734,-0.611740,...,-0.204105,-0.235082,-0.269715,-0.134629,-0.163342,-0.508718,-0.135585,-0.397808,-0.188538,-0.219639
672,-1.474754,-0.354830,0.248374,0.886110,0.343271,-0.600870,2.551846,-2.056483,-1.216225,1.366836,...,-0.203527,-0.234550,-0.269257,-0.133805,-0.162630,-0.508450,-0.134797,-0.397498,-0.187937,-0.219131
723,-0.597235,0.059911,0.248374,0.490712,0.343271,0.717183,-0.872381,-0.360438,-0.771996,-0.021677,...,-0.204054,-0.235035,-0.269674,-0.134556,-0.163279,-0.508695,-0.135515,-0.397781,-0.188485,-0.219594
121,-1.574472,-0.354830,0.248374,-0.942320,0.343271,-1.055371,0.027236,2.110704,1.171508,2.097286,...,-0.202305,-0.233424,-0.268288,-0.132063,-0.161125,-0.507884,-0.133133,-0.396844,-0.186668,-0.218057


In [75]:
lgclf = LogisticRegression(random_state=42, fit_intercept = True, solver = "liblinear", penalty ="l1").fit(X_train, y_train) 

In [76]:
print(cross_val_score(lgclf, X_train, y_train, cv=10).mean())

0.9282200869366278


In [77]:
print("Accuracy Score:",lgclf.score(X_test, y_test))

Accuracy Score: 0.9038461538461539


In [78]:
coeff_df = pd.DataFrame(X_data.columns)
coeff_df.columns = ['Features']
coeff_df["Coefficient"] = pd.Series(lgclf.coef_[0])

# preview
coeff_df.sort_values(by='Coefficient', inplace=True)
df_f0 = coeff_df[coeff_df["Coefficient"] == 0]
df_feature = coeff_df[coeff_df["Coefficient"] != 0]

In [81]:
df_feature

,Features,Coefficient
107,neg_sent_TA,-2.607553
164,t11,-1.021074
153,Conjunction.Start,-0.979487
86,Exclam,-0.901988
32,anx,-0.854058
...,...,...
11,i,0.829521
100,valence..EL._TA,0.858910
135,Hello,0.896843
141,Negative.Emotion,1.035047


In [80]:
df_f0

,Features,Coefficient
96,concreteness_TA,0.0
115,i_TA,0.0
5,WPS,0.0
95,valence..ANEW._TA,0.0
101,SENT_TA,0.0
...,...,...
77,assent,0.0
65,relativ,0.0
75,informal,0.0
69,work,0.0
